In [1]:
import keras
from keras.layers import Input, Dense, Dot, Softmax
from keras.models import Model

import numpy as np

Using TensorFlow backend.


We will make 6 dimensional clothing vectors, and train the way to find transform them into the 2 dimensional "shoe vector" space

In [2]:
CLOTHING_DIM = 6
SHOE_DIM = 2

clothe_input = Input(shape=[CLOTHING_DIM], name='clothing_input')
clothe_transform = Dense(SHOE_DIM, activation='linear', name='clothe_to_shoe')(clothe_input)

shoe_input = Input(shape=[SHOE_DIM], name='shoe_input')

prod = Dot(axes=1, name='DotProduct')([clothe_transform, shoe_input])
pred = Dense(1, name='Prediction',  activation='sigmoid')(prod)

model = Model(inputs=[clothe_input, shoe_input], outputs=[pred])
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

Now let's make up some data to fit (4 examples of clothes are shoe pairings).

In the real example, it would be "clothing_vector shoe_vector" and the corresponding rating. Note the same "clothing vector" and "shoe_vector" can appear on multiple rows.

In [3]:
train_clothe = np.array([[0,0,1,1,-1,-1], [1,0.2,0,0.2,-1,-1], [1,1,1,0.3,-1,0], [1,0.3,0.4,0.2,0.2,-1]]).astype('float')
train_shoe = np.array([[1,1],[0.2,1],[0.3,0.2],[-0.1,0.2]]).astype('float')
ratings = np.array([1,0,1,0])

Fit the model

In [4]:
model.fit([train_clothe, train_shoe], ratings, epochs=8000, verbose=0)

In [5]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
clothing_input (InputLayer)     (None, 6)            0                                            
__________________________________________________________________________________________________
clothe_to_shoe (Dense)          (None, 2)            14          clothing_input[0][0]             
__________________________________________________________________________________________________
shoe_input (InputLayer)         (None, 2)            0                                            
__________________________________________________________________________________________________
DotProduct (Dot)                (None, 1)            0           clothe_to_shoe[0][0]             
                                                                 shoe_input[0][0]                 
__________

Notice how close these are to the made up ratings

In [6]:
model.predict([train_clothe, train_shoe])

array([[1.000000e+00],
       [8.825822e-05],
       [9.998654e-01],
       [7.974839e-04]], dtype=float32)

## Now get the transformation from clothing space to shoe space

Let's get just the transformation that takes a dress vector (dim `CLOTHING_DIM`=6) and returns a shoe vector (dim `SHOE_DIM`=2)

In [7]:
dress_to_shoe = Model(inputs=[clothe_input], outputs=[clothe_transform])
shoe_vectors = dress_to_shoe.predict(train_clothe)
shoe_vectors

array([[10.667187 , -4.118615 ],
       [10.79912  , -4.405012 ],
       [13.384578 , -5.1150484],
       [ 9.448515 , -3.341847 ]], dtype=float32)

In [8]:
# This is the dot product, check results from above
np.sum(shoe_vectors * train_shoe, axis=1)

array([ 6.54857159, -2.24518814,  2.99236364, -1.61322088])

We can also extract the weights from that layer directly and build the transformation

$$\text{shoe vector} = Ax + b$$

where $x$ is the `dress_vector`

In [9]:
A, b = model.get_layer(name='clothe_to_shoe').get_weights()
A

array([[ 3.280788  , -0.60493165],
       [ 2.7914267 , -2.099624  ],
       [ 1.6551527 ,  0.33186847],
       [ 2.5649836 , -1.3379103 ],
       [-1.9098415 ,  0.95031685],
       [-1.5593354 ,  0.77158546]], dtype=float32)

Let's manually construct the clothes vectors

In [10]:
output2 = train_clothe @ A + b
output2

array([[10.66718698, -4.11861527],
       [10.79912069, -4.40501194],
       [13.3845777 , -5.11504817],
       [ 9.44851458, -3.34184673]])

Let's show this is the same (up to rounding) as the `shoe_vectors` we found the hard way earlier

In [11]:
np.allclose(output2, shoe_vectors)

True

In [12]:
# numerically
output2 - shoe_vectors

array([[ 2.38418579e-07, -1.19209290e-07],
       [ 7.39097596e-07,  1.90734863e-07],
       [-4.76837165e-08,  2.38418579e-07],
       [-3.57627869e-07,  2.14576721e-07]])

In [13]:
# Hopefully in your real model these numbers will be positive, especially the first one
model.get_layer(name='Prediction').get_weights()

[array([[3.484206]], dtype=float32), array([-1.5124576], dtype=float32)]

## New clothes

The process for new clothes:
1. Get the clothe_vector using the pre-existing model
2. Use `dress_to_shoe.predict` to get the corresponding shoe vector
3. Now look in the neighborhood of those returned shoe vectors. Use a neighborhood model here to find similar shoes